In [11]:
# Cell 3: Set up Django environment
import os
import sys
import django
import torch
from django.conf import settings
from django.core.handlers.wsgi import WSGIHandler
from django.core.wsgi import get_wsgi_application
from django.http import HttpResponse, JsonResponse
from django.urls import path
from django.template import Template, Context

if not settings.configured:
    settings.configure(
        DEBUG=True,
        ROOT_URLCONF=__name__,
        SECRET_KEY='your-secret-key-here',
        TEMPLATES=[{
            'BACKEND': 'django.template.backends.django.DjangoTemplates',
            'DIRS': [],
            'OPTIONS': {'debug': True},
        }],
        INSTALLED_APPS=[
            'django.contrib.staticfiles',
            'rest_framework',
        ],
        STATIC_URL='/static/',
        REST_FRAMEWORK={
            'DEFAULT_PERMISSION_CLASSES': [],
            'DEFAULT_AUTHENTICATION_CLASSES': [],
        }
    )
    django.setup()

# Cell 4: Define and load the PyTorch model
import torch.nn as nn
import cv2
import numpy as np
import tempfile

class CNNModel(nn.Module):
    def __init__(self, input_shape, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(input_shape[0], 48, kernel_size=7, stride=1, padding=3)
        self.bn1 = nn.BatchNorm2d(48)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.25)
        self.conv2 = nn.Conv2d(48, 128, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 56 * 56, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.dropout2 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        print("Input shape:", x.shape)  # Debug
        x = torch.relu(self.bn1(self.conv1(x)))
        print("After conv1+bn1:", x.shape)
        x = self.pool1(x)
        print("After pool1:", x.shape)
        x = self.dropout1(x)
        x = torch.relu(self.bn2(self.conv2(x)))
        print("After conv2+bn2:", x.shape)
        x = self.pool2(x)
        print("After pool2:", x.shape)
        x = self.flatten(x)
        print("After flatten:", x.shape)
        x = torch.relu(self.bn3(self.fc1(x)))
        print("After fc1+bn3:", x.shape)
        x = self.dropout2(x)
        x = self.fc2(x)
        print("Output shape:", x.shape)
        return x

device = torch.device("cpu")
model = CNNModel(input_shape=(1, 224, 224), num_classes=2).to(device)
model.load_state_dict(torch.load("C:\\Users\\M lapan\\Downloads\\best_model.pth", map_location=device))  
model.eval()  # Ensure evaluation mode

# Cell 5: Define preprocessing and views
def preprocess_frame(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = cv2.resize(frame, (224, 224)) / 255.0
    frame = torch.tensor(frame, dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # Shape: [1, 1, 224, 224]
    return frame.to(device)

def process_video(video):
    if hasattr(video, 'temporary_file_path'):
        video_path = video.temporary_file_path()
    else:
        with tempfile.NamedTemporaryFile(delete=False, suffix='.mp4') as temp_file:
            for chunk in video.chunks():
                temp_file.write(chunk)
            video_path = temp_file.name

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        if not hasattr(video, 'temporary_file_path'):
            os.unlink(video_path)
        return None

    predictions = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        input_tensor = preprocess_frame(frame)
        with torch.no_grad():
            output = model(input_tensor)
            pred = torch.argmax(output, dim=1).item()
            predictions.append("Shopping" if pred == 1 else "Non-Shopping")
    cap.release()

    if not hasattr(video, 'temporary_file_path'):
        os.unlink(video_path)

    return predictions

def index(request):
    if request.method == "POST" and request.FILES.get("video"):
        video = request.FILES["video"]
        predictions = process_video(video)
        if predictions is None:
            return HttpResponse("Error: Could not process video.")

        template = Template("""
            <h1>Classification Results</h1>
            <ul>
                {% for pred in predictions %}
                    <li>{{ pred }}</li>
                {% endfor %}
            </ul>
            <a href="/">Back</a>
        """)
        context = Context({"predictions": predictions})
        return HttpResponse(template.render(context))

    return HttpResponse("""
        <h1>Video Classifier</h1>
        <form method="post" enctype="multipart/form-data">
            <input type="file" name="video" accept="video/*">
            <button type="submit">Classify</button>
        </form>
    """)

# API view
from rest_framework.decorators import api_view
from rest_framework.response import Response
from rest_framework import status

@api_view(['POST'])
def classify_video_api(request):
    if 'video' not in request.FILES:
        return Response({"error": "No video file provided"}, status=status.HTTP_400_BAD_REQUEST)
    
    video = request.FILES["video"]
    predictions = process_video(video)
    if predictions is None:
        return Response({"error": "Could not process video"}, status=status.HTTP_400_BAD_REQUEST)
    
    return Response({"predictions": predictions}, status=status.HTTP_200_OK)

# Cell 6: Set up URLs and start server
urlpatterns = [
    path('', index),
    path('api/classify/', classify_video_api),
]
application = get_wsgi_application()

from wsgiref.simple_server import make_server
import threading

def run_server():
    httpd = make_server('localhost', 8000, application)
    print("Serving on http://localhost:8000")
    httpd.serve_forever()

thread = threading.Thread(target=run_server)
thread.start()

Serving on http://localhost:8000
